In [1]:
import polyline
import datetime
import googlemaps
from googlemaps import convert
import urllib.request
import json
import gmaps
from geo import GeoLocation
import numpy as np
#from geopy.distance import geodesic
from math import pi,sqrt,sin,cos,atan2

#------------------------------------------------------------------------------------------------------------
def get_polyline_points(driver_start,driver_end):
    '''Given a start and end locations, This function returns a list containing a set of Geo coordinates of 
    the driver's path
    
    '''
    endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
    api_key = 'enter your API Key
  # origin = input ('where are you?').replace(' ','+')
  # destination = input ('where to go ?').replace(' ','+')
    origin =(driver_start).replace(' ','+')
    destination =(driver_end).replace(' ','+')
    nav_request ='origin={}&destination={}&key={}'.format(origin,destination,api_key)
    request = endpoint + nav_request
    response = urllib.request.urlopen(request).read()
    directions = json.loads(response.decode('utf-8'))
    poldriver=directions['routes'][0]['overview_polyline']['points'] #get encrypted polyline of the driver  
    dpol=polyline.decode(poldriver) #decode polyline to get list of points
    return dpol
#------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------


#----------------------------------------------------------------------------------------------------
def get_random_test_trips(passenger_start,passenger_end,Numdrivers,distance_from_target):
    '''
    This function generates "Nuldrivers" tris within the area delimited by the Start and the end point of the passenger
    
    '''
    endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
    api_key = 'enter yout API key'
    # origin = input ('where are you?').replace(' ','+')
    # destination = input ('where to go ?').replace(' ','+')
    origin = (passenger_start).replace(' ','+')
    destination = (passenger_end).replace(' ','+')
    nav_request ='origin={}&destination={}&key={}'.format(origin,destination,api_key)
    request = endpoint + nav_request
    response = urllib.request.urlopen(request).read()
    directions = json.loads(response.decode('utf-8'))
    Pass_home_lat=directions['routes'][0]['legs'][0]['start_location']['lat']
    Pass_home_lng=directions['routes'][0]['legs'][0]['start_location']['lng']
    Pass_work_lat=directions['routes'][0]['legs'][0]['end_location']['lat']
    Pass_work_lng=directions['routes'][0]['legs'][0]['end_location']['lng']
    Pass_home_coord = (Pass_home_lat,Pass_home_lng)
    Pass_work_coord = (Pass_work_lat,Pass_work_lng)
    #------bounds determination ---------------------------------------------------------------------------
    bound=directions['routes'][0]['bounds']
    corner1lat=bound['southwest']['lat']
    corner1lng=bound['southwest']['lng']
    corner2lat=bound['northeast']['lat']
    corner2lng=bound['northeast']['lng']
    mark=[(corner1lat,corner1lng),(corner2lat,corner2lng)] # get the bounds NE-SW of the passenger locations
    #get bounds of the focus area---geoloc lib------!!we need to get an area large than the bounds given above!!
    loc1 = GeoLocation.from_degrees(corner1lat, corner1lng)
    loc2 = GeoLocation.from_degrees(corner2lat, corner2lng)
    distance = distance_from_target  #we cover the area as SW=SW+ x km and NE=NE+ x km
    SW_locSW, NE_locSW = loc1.bounding_locations(distance)
    SW_locNE, NE_locNE = loc2.bounding_locations(distance)
    first=(str(SW_locSW)).split()
    second=(str(NE_locNE)).split()
    SWcornerlat=float((''.join(i for i in first[0] if (i.isdigit() or i=='.'))))
    SWcornerlng=float((''.join(i for i in first[1] if (i.isdigit() or i=='.'))))
    NEcornerlat=float((''.join(i for i in second[0] if (i.isdigit() or i=='.'))))
    NEcornerlng=float((''.join(i for i in second[1] if (i.isdigit() or i=='.'))))
    drivers_number=Numdrivers
    scale = np.random.random_sample(4*drivers_number)
    latitudes=(scale*(NEcornerlat-SWcornerlat))+SWcornerlat
    langitudes=(scale*(NEcornerlng-SWcornerlng))+SWcornerlng
    langitudes=list(langitudes)
    latitudes=list(latitudes)
    coords=[]
    a=0
    b=len(latitudes)
    b-=1
    while a<=b:
        coords.append(latitudes[a])
        coords.append(langitudes[a])
        a+=1
    coco=list()
    a=0
    b=len(latitudes)
    while a<=(b-1):
        coco.append((coords[a].item(),coords[a+1].item()))
        a+=2
    start_points = coco[:len(coco)//2]#departure locations in the area 
    end_points = coco[len(coco)//2:]#arrival locations in the area 
    #start_points,end_points = filter_driver_data(start_points,end_points,Pass_home_coord,Pass_work_coord)
    
    drivers_polylines_data=list()
    a=0
    b=len(start_points)
    while a<=(b-1):
        Dhome=str(start_points[a][0])+','+str(start_points[a][1])
        Dwork=str(end_points[a][0])+','+str(end_points[a][1])
        drivers_polylines_data.append(get_polyline_points(Dhome,Dwork))
        a+=1
    return (Pass_home_coord,Pass_work_coord,drivers_polylines_data,SWcornerlat,SWcornerlng,NEcornerlat,NEcornerlng,len(start_points))

#---------------get distance between 2 positions---------------------------------------------------------------------------------------------
def haversine(pos1, pos2):
    lat1 = float(pos1[0])
    long1 = float(pos1[1])
    lat2 = float(pos2[0])
    long2 = float(pos2[1])

    degree_to_rad = float(pi / 180.0)

    d_lat = (lat2 - lat1) * degree_to_rad
    d_long = (long2 - long1) * degree_to_rad

    a = pow(sin(d_lat / 2), 2) + cos(lat1 * degree_to_rad) * cos(lat2 * degree_to_rad) * pow(sin(d_long / 2), 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    km = 6367 * c
    #mi = 3956 * c
    return km
#------------------------------------------------------------------------------------------------------------

In [2]:
home="17,Rue Andre Ampere, Ariana, Tunisie"
work="4, Rue Mosbah Jarboua, ben Arous, Tunisie"
#--------------------------------------------------------------------------------
coord1,coord2,poly,a,b,c,d,effect_drivers= get_random_test_trips(home,work,100,10)

In [14]:
def get_distance_output(driverdata,passenger_home=coord1,passenger_work=coord2):
    distlist1=list()
    distlist2=list()
   
    i=0
    l=len(driverdata)
    while i<=(l-1):
        dis1=haversine(driverdata[i],coord1)
        distlist1.append(dis1)
        i+=1
    (hmin,s) = min((v,s) for s,v in enumerate(distlist1))
    j=s
    while j<=(l-1):
            dis2=haversine(driverdata[j],coord2)
            distlist2.append(dis2)
            j+=1
    return (min(distlist1)+min(distlist2))
    


def get_pickupPoint_coord(driverdata,passenger_home=coord1,passenger_work=coord2):
    distlist1=list()
    distlist2=list()
    a=0
    b=len(driverdata)
    while a<=(b-1):
        dis1=haversine(driverdata[a],coord1)
        dis2=haversine(driverdata[a],coord2)
        distlist1.append(dis1)
        distlist2.append(dis2)
        a+=1
    x=0
    i=0
    w=0
    q=0
    while x<=(b-1):
        if (distlist1[x]==min(distlist1)):
            w=x
        x+=1
    while i<=(b-1):
        if (distlist2[i]==min(distlist2)):
            q=i
        i+=1
    return (driverdata[w],driverdata[q])
#-----------------------------------------------------------------
def sort_lesswalking (n,coord1=coord1,coord2=coord2):
    
    pick,drop=get_pickupPoint_coord(n)
    client=googlemaps.Client('Google API Key')        
    x=googlemaps.client.distance_matrix(client,coord1,pick,mode="walking")
    y=googlemaps.client.distance_matrix(client,drop,work,mode="walking")
    home_to_pick_dist=(x['rows'][0]['elements'][0]['distance']['value'])/1000
    drop_to_work_dist=(y['rows'][0]['elements'][0]['distance']['value'])/1000
    return (home_to_pick_dist+drop_to_work_dist)

In [16]:
n=sorted(poly, key=get_distance_output)


In [62]:
n=sorted(n, key=sort_lesswalking)

In [17]:
def visualize_data(rank,n,coord1,coord2): 
    '''
    This function permits the vizualisaton of the suitable rides for the passenger.
    '''
    gmaps.configure(api_key='enter API KEY')
    fig = gmaps.figure()
    rank-=1
    pick,drop=get_pickupPoint_coord(n[rank])
    marker_locations1=[coord1]
    marker_locations2=[coord2]
    marker_locations3=[pick]
    marker_locations4=[drop]
    #------------drawings (box)----------------------------------------
    box = gmaps.drawing_layer(features=[gmaps.Polygon([(a, b),(c,b),(c,d), (a,d)],fill_color='red')])
    #-------------directions-------------------------------------------------------------------
    bestroute = gmaps.directions_layer(n[0][0], n[0][len(n[0])-1],stroke_color='red',stroke_weight=2)
    best_by_passenger = gmaps.directions_layer(n[0][0], n[0][len(n[0])-1],waypoints=[coord1,coord2],stroke_color='blue',stroke_weight=1)
    best_nth_route = gmaps.directions_layer(n[rank][0], n[rank][len(n[rank])-1])
    home_to_pickup= gmaps.directions_layer(coord1,pick,travel_mode='WALKING',show_markers=False,stroke_color='red',stroke_weight=2)
    drop_to_work=gmaps.directions_layer(drop,coord2,travel_mode='WALKING',show_markers=False,stroke_color='red',stroke_weight=2)
    #-------------------markers----------------------------------------------------------------
    markerhome = gmaps.marker_layer(marker_locations1,info_box_content='passenger home')
    markerwork = gmaps.marker_layer(marker_locations2,info_box_content='passenger work')
    marker_pickup_point = gmaps.symbol_layer(marker_locations3,fill_color='green',scale=4,info_box_content='pickup_point')
    marker_drop_point = gmaps.symbol_layer(marker_locations4,fill_color='green',scale=4,info_box_content='drop_point')
    #-----------------adding layers-----------------------------------
    fig.add_layer(best_nth_route)
    fig.add_layer(markerhome)
    fig.add_layer(markerwork)
    fig.add_layer(marker_pickup_point)
    fig.add_layer(marker_drop_point)
    #fig.add_layer(bestroute)
    fig.add_layer(box)
    fig.add_layer(home_to_pickup)
    fig.add_layer(drop_to_work)
    #fig.add_layer(best1)
    
    #---------------------numeric data-------------------------------
    client=googlemaps.Client('API Key)
    x=googlemaps.client.distance_matrix(client,coord1,pick,mode="walking")
    y=googlemaps.client.distance_matrix(client,drop,work,mode="walking")
    trip_data=googlemaps.client.distance_matrix(client,pick,drop,mode="driving")
    home_to_pick_dist=(x['rows'][0]['elements'][0]['distance']['value'])/1000
    home_to_pick_time=str(datetime.timedelta(seconds=(x['rows'][0]['elements'][0]['duration']['value'])))
    drop_to_work_dist=(y['rows'][0]['elements'][0]['distance']['value'])/1000
    drop_to_work_time=str(datetime.timedelta(seconds=(y['rows'][0]['elements'][0]['duration']['value'])))
    pick_to_drop_dist=(trip_data['rows'][0]['elements'][0]['distance']['value'])/1000
    pick_to_drop_time=str(datetime.timedelta(seconds=(trip_data['rows'][0]['elements'][0]['duration']['value'])))
    totaldistance=home_to_pick_dist+drop_to_work_dist+pick_to_drop_dist
    totaltime=str(datetime.timedelta(seconds=(trip_data['rows'][0]['elements'][0]['duration']['value']+y['rows'][0]['elements'][0]['duration']['value']+x['rows'][0]['elements'][0]['duration']['value'])))
    print('I generated',len(n),'drives suitable for you\n')
    print('this is the',rank+1,'st/th less walking best driving suggestion\n')
    print('you will walk',home_to_pick_dist,'km from home to pickup point for a duration of',home_to_pick_time,'\n')
    print('you will be drived',pick_to_drop_dist,'km from pickup to drop point for a duration of',pick_to_drop_time,'\n')
    print('you will walk again',drop_to_work_dist,'km from drop point to work for a duration of',drop_to_work_time,'\n')
    print('the by foot distance is',home_to_pick_dist+drop_to_work_dist,'km \n')
    print('the whole distance is',totaldistance,'km for a duration of',totaltime)
    return fig

In [20]:
# Here is an exemple
visualize_data(3,n,coord1,coord2)

I generated 100 drives suitable for you

this is the 3 st/th less walking best driving suggestion

you will walk 4.019 km from home to pickup point for a duration of 0:49:49 

you will be drived 10.587 km from pickup to drop point for a duration of 0:09:28 

you will walk again 3.707 km from drop point to work for a duration of 0:44:18 

the by foot distance is 7.726 km 

the whole distance is 18.313 km for a duration of 1:43:35


Figure(layout=FigureLayout(height='420px'))